In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [6]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [7]:
engine.execute("SELECT * FROM measurement order by date ").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2725, 'USC00513117', '2010-01-01', 0.28, 67.0),
 (9008, 'USC00518838', '2010-01-01', 0.21, 72.0),
 (9519, 'USC00519523', '2010-01-01', 0.15, 75.0),
 (12188, 'USC00519281', '2010-01-01', 0.15, 70.0),
 (14960, 'USC00511918', '2010-01-01', 0.05, 66.0),
 (16939, 'USC00516128', '2010-01-01', 0.14, 73.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (2726, 'USC00513117', '2010-01-02', 0.0, 65.0),
 (9009, 'USC00518838', '2010-01-02', 0.02, 73.0),
 (9520, 'USC00519523', '2010-01-02', 0.0, 75.0),
 (12189, 'USC00519281', '2010-01-02', 0.0, 62.0),
 (14961, 'USC00511918', '2010-01-02', 0.0, 70.0),
 (16940, 'USC00516128', '2010-01-02', 0.0, 61.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (2727, 'USC00513117', '2010-01-03', 0.0, 75.0),
 (9010, 'USC00518838', '2010-01-03', 0.0, 67.0),
 (9521, 'USC00519523', '2010-01-03', 0.0, 77.0),
 (12190, 'USC00519281', '2010-01-03', 0.0, 74.0),
 (14962, 'USC00511918', '2010-01-03', 0.0, 75.0),
 (16941, 'USC

In [8]:
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [9]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables
Base.classes.keys()

['measurement', 'station']

In [10]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [11]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [12]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

most_recent_date = dt.datetime.strptime(most_recent_date[0], '%Y-%m-%d')

# Calculate the date one year from the last date in data set.
twelve_months_ago = most_recent_date - dt.timedelta(days=365)

print(f"Lookup Date: {twelve_months_ago}")

Lookup Date: 2016-08-23 00:00:00


In [13]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Perform a query to retrieve the data and precipitation scores
sel = [Measurement.date, func.sum(Measurement.prcp)]
prcp_scores = session.query(*sel).filter(Measurement.date>=twelve_months_ago, Measurement.prcp!="None").group_by(Measurement.date).order_by(Measurement.date).all()

prcp_scores

[('2016-08-24', 9.329999999999998),
 ('2016-08-25', 0.54),
 ('2016-08-26', 0.09999999999999999),
 ('2016-08-27', 0.31999999999999995),
 ('2016-08-28', 3.0999999999999996),
 ('2016-08-29', 1.46),
 ('2016-08-30', 0.07),
 ('2016-08-31', 3.1799999999999997),
 ('2016-09-01', 0.03),
 ('2016-09-02', 0.25),
 ('2016-09-03', 1.27),
 ('2016-09-04', 1.3800000000000001),
 ('2016-09-05', 0.33999999999999997),
 ('2016-09-06', 1.23),
 ('2016-09-07', 2.0),
 ('2016-09-08', 0.4600000000000001),
 ('2016-09-09', 1.0699999999999998),
 ('2016-09-10', 1.4),
 ('2016-09-11', 1.25),
 ('2016-09-12', 1.54),
 ('2016-09-13', 2.71),
 ('2016-09-14', 14.280000000000001),
 ('2016-09-15', 4.96),
 ('2016-09-16', 0.8200000000000001),
 ('2016-09-17', 0.6799999999999999),
 ('2016-09-18', 0.53),
 ('2016-09-19', 0.32),
 ('2016-09-20', 1.03),
 ('2016-09-21', 1.17),
 ('2016-09-22', 1.44),
 ('2016-09-23', 1.57),
 ('2016-09-24', 0.28),
 ('2016-09-25', 0.09),
 ('2016-09-26', 1.67),
 ('2016-09-27', 1.3599999999999999),
 ('2016-09-28

In [14]:
# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_scores_df = pd.DataFrame(prcp_scores, columns=['Date', 'Precipitation Score'])
prcp_scores_df.set_index('Date', inplace=True)

# Sort the dataframe by date
prcp_scores_df = prcp_scores_df.sort_index()

prcp_scores_df

Precipitation Score
Date                           
2016-08-24                 9.33
2016-08-25                 0.54
2016-08-26                 0.10
2016-08-27                 0.32
2016-08-28                 3.10
...                         ...
2017-08-19                 0.09
2017-08-20                 0.01
2017-08-21                 0.58
2017-08-22                 0.50
2017-08-23                 0.53

[365 rows x 1 columns]

In [24]:
# Use Pandas Plotting with Matplotlib to plot the data
ax = prcp_scores_df.plot.bar()

for i, t in enumerate(ax.get_xticklabels()):
    if (i % 30) != 0:
        t.set_visible(False)

ax.set_ylabel("Inches")
plt.tight_layout()
plt.show()

In [25]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_scores_df.describe()

Precipitation Score
count           365.000000
mean              0.974164
std               1.776466
min               0.000000
25%               0.050000
50%               0.400000
75%               1.080000
max              14.280000

# Exploratory Station Analysis

In [17]:
# Design a query to calculate the total number stations in the dataset


In [18]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [19]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [20]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [21]:
# Close Session
session.close()